<a href="https://colab.research.google.com/github/peremartra/Large-Language-Model-Notebooks-Course/blob/main/4-Evaluating%20LLMs/4_3_evaluating_rag_giskard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div>
    <h1>Large Language Models Projects</a></h1>
    <h3>Apply and Implement Strategies for Large Language Models</h3>
    <h2>4.3-Evaluating LLMs with other LLMs.
  </h2>
    <h3>Evaluating a RAG solution with Giskard</h3>
</div>

by [Pere Martra](https://www.linkedin.com/in/pere-martra)

____________
Models: OpenAI.

Colab Environment: CPU-High RAM

Keys:
* LangChain.
* RAG.
* Giskard.
___________

This is the unofficial repository for the book:
        <a href="https://amzn.to/4eanT1g"> <b>Large Language Models:</b> Apply and Implement Strategies for Large Language Models</a> (<a href="https://link.springer.com/book/10.1007/979-8-8688-0515-8">Apress</a>).
        The book is based on the content of this repository, but the notebooks are being updated, and I am incorporating new examples and chapters.
        If you are looking for the official repository for the book, with the original notebooks, you should visit the
        <a href="https://github.com/Apress/Large-Language-Models-Projects">Apress repository</a>, where you can find all the notebooks in their original format as they appear in the book.

In the final part of this notebook, you will find the necessary code to evaluate the suitability of the responses provided by the Agent using Giskard.

#Installing libraries & Loading Dataset

In [1]:
!pip install -q langchain==0.3.2
!pip install -q langchain-openai==0.2.1
!pip install -q langchainhub==0.1.21
!pip install -q datasets==3.0.1
!pip install -q chromadb==0.5.11
!pip install -q openai==1.51.0
!pip install -q langchain-community==0.3.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.5/383.5 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.

We will download the dataset from the Hugging Face datasets library. It's a dataset with information about diseases.

In [2]:
from datasets import load_dataset

data = load_dataset("keivalya/MedQuad-MedicalQnADataset", split='train')


README.md:   0%|          | 0.00/233 [00:00<?, ?B/s]

medDataset_processed.csv:   0%|          | 0.00/22.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16407 [00:00<?, ? examples/s]

In [3]:
data = data.to_pandas()
data.head(10)

,qtype,Question,Answer
0,susceptibility,Who is at risk for Lymphocytic Choriomeningiti...,LCMV infections can occur after exposure to fr...
1,symptoms,What are the symptoms of Lymphocytic Choriomen...,LCMV is most commonly recognized as causing ne...
2,susceptibility,Who is at risk for Lymphocytic Choriomeningiti...,Individuals of all ages who come into contact ...
3,exams and tests,How to diagnose Lymphocytic Choriomeningitis (...,"During the first phase of the disease, the mos..."
4,treatment,What are the treatments for Lymphocytic Chorio...,"Aseptic meningitis, encephalitis, or meningoen..."
5,prevention,How to prevent Lymphocytic Choriomeningitis (L...,LCMV infection can be prevented by avoiding co...
6,information,What is (are) Parasites - Cysticercosis ?,Cysticercosis is an infection caused by the la...
7,susceptibility,Who is at risk for Parasites - Cysticercosis? ?,Cysticercosis is an infection caused by the la...
8,exams and tests,How to diagnose Parasites - Cysticercosis ?,"If you think that you may have cysticercosis, ..."
9,treatment,What are the treatments for Parasites - Cystic...,Some people with cysticercosis do not need to ...


In [6]:
data = data[0:100]

As you can see, the medical information in the dataset is well-organized, and to someone like me, who is not an expert in the field, it appears to be quite valuable. This information could be a useful addition to any general medicine book to support primary care doctors.

Load the langchain libraries to load the document.

In [7]:
from langchain.document_loaders import DataFrameLoader
from langchain.vectorstores import Chroma

The Document is in the Answer column, and the others columns are Metadata.

In [8]:
df_loader = DataFrameLoader(data, page_content_column="Answer")


In [9]:
df_document = df_loader.load()
display(df_document[:2])

[Document(metadata={'qtype': 'susceptibility', 'Question': 'Who is at risk for Lymphocytic Choriomeningitis (LCM)? ?'}, page_content='LCMV infections can occur after exposure to fresh urine, droppings, saliva, or nesting materials from infected rodents.  Transmission may also occur when these materials are directly introduced into broken skin, the nose, the eyes, or the mouth, or presumably, via the bite of an infected rodent. Person-to-person transmission has not been reported, with the exception of vertical transmission from infected mother to fetus, and rarely, through organ transplantation.'),
 Document(metadata={'qtype': 'symptoms', 'Question': 'What are the symptoms of Lymphocytic Choriomeningitis (LCM) ?'}, page_content='LCMV is most commonly recognized as causing neurological disease, as its name implies, though infection without symptoms or mild febrile illnesses are more common clinical manifestations. \n                \nFor infected persons who do become ill, onset of sympt

We can chunk the documents. The size to which we want to split the document is a design decision. The larger it is, the larger the prompt will be, and the slower the Model's response process.

We also need to consider the maximum prompt size and ensure that the document does not exceed it.

In [10]:
from langchain.text_splitter import CharacterTextSplitter

In [11]:
text_splitter = CharacterTextSplitter(chunk_size=1250, chunk_overlap=100)
texts = text_splitter.split_documents(df_document)


These warnings we see are because it can't perform the partition of the required size. This is because it waits for a page break to divide the text and does so when possible.

In [12]:
first_doc = texts[1]
print(first_doc.page_content)

LCMV is most commonly recognized as causing neurological disease, as its name implies, though infection without symptoms or mild febrile illnesses are more common clinical manifestations. 
                
For infected persons who do become ill, onset of symptoms usually occurs 8-13 days after exposure to the virus as part of a biphasic febrile illness. This initial phase, which may last as long as a week, typically begins with any or all of the following symptoms: fever, malaise, lack of appetite, muscle aches, headache, nausea, and vomiting. Other symptoms appearing less frequently include sore throat, cough, joint pain, chest pain, testicular pain, and parotid (salivary gland) pain. 
                
Following a few days of recovery, a second phase of illness may occur. Symptoms may consist of meningitis (fever, headache, stiff neck, etc.), encephalitis (drowsiness, confusion, sensory disturbances, and/or motor abnormalities, such as paralysis), or meningoencephalitis (inflammation 

### Initialize the Embedding Model and Vector DB

We load the text-embedding-ada-002 model from OpenAI.

In [13]:
from getpass import getpass
import os
if not 'OPENAI_API_KEY' in os.environ:
  os.environ["OPENAI_API_KEY"] = getpass("OpenAI API Key: ")

OpenAI API Key: ··········


In [14]:
from langchain_openai import OpenAIEmbeddings

model_name = 'text-embedding-ada-002'
#model_name = 'text-embedding-3-small'

embed = OpenAIEmbeddings(
    model=model_name,
    #openai_api_key=OPENAI_API_KEY
)

The execution of this cell may take 3 to 5 minutes. If you want it to be faster, you can reduce the number of records in the dataset.

In [15]:
directory_cdb = '/content/drive/MyDrive/chromadb'
chroma_db = Chroma.from_documents(
    df_document, embed, persist_directory=directory_cdb
)

We are going to create three objects.

* The language model, which can be any of those from OpenAI, the most common being gpt-3.5.
* The memory, responsible for keeping the prompt with all the necessary history.
* The retrieval, used to obtain information stored in ChromaDB.

In [16]:
from langchain.chat_models import ChatOpenAI
from langchain_openai import OpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chains import RetrievalQA

llm=OpenAI(#openai_api_key=OPENAI_API_KEY,
           temperature=0.0)

conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=4, #Number of messages stored in memory
    return_messages=True #Must return the messages in the response.
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=chroma_db.as_retriever()
)

<ipython-input-16-b1e61a5f64ec>:9: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  conversational_memory = ConversationBufferWindowMemory(


We can try the isolated Retrieval to see if the information it returns is relevant.




In [17]:
qa.invoke("What is the main symptom of LCM?")

{'query': 'What is the main symptom of LCM?',
 'result': ' The main symptom of LCM is a biphasic febrile illness, which includes symptoms such as fever, malaise, lack of appetite, muscle aches, headache, nausea, and vomiting.'}

Perfect! The information returned is exactly what we desired.

## Creating the Agent.

In [18]:
from langchain.agents import Tool, AgentExecutor
DESCRIPTION = 'use this tool when answering medical knowledge queries to get more information about the topic'
#DESCRIPTION = 'Use this tool if any part of the QUERY or CHAT HISTORY history discusses concepts related to medicine or diseases to get more information about the topic'

#Defining the list of tool objects to be used by LangChain.
tools = [
    Tool(
        name='Medical KB',
        func=qa.run,
        description=(
            DESCRIPTION
        )
    )
]

In [19]:
from langchain.agents import initialize_agent, create_react_agent
from langchain import hub

prompt = hub.pull("hwchase17/react-chat")
agent = create_react_agent(
    #agent='chat-conversational-react-description',
    tools=tools,
    llm=llm,
    prompt=prompt,
    #verbose=True,
    #max_iterations=3,
    #early_stopping_method='generate'
)

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [20]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent,
                               tools=tools,
                               verbose=True,
                               memory=conversational_memory,
                               max_iterations=30,
                               max_execution_time=600,
                               #early_stopping_method='generate',
                               handle_parsing_errors=True
                               )

### Using the Conversational Agent

To make queries we simply call the `agent` directly.

First i will try a order not related to the Medical field.

In [21]:
agent_executor.invoke({"input": "Give me the area of square of 2x2"})



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: Medical KB
Action Input: Area of square I don't know.Do I need to use a tool? No
Final Answer: The area of a square with sides of 2 units is 4 square units.

> Finished chain.


{'input': 'Give me the area of square of 2x2',
 'chat_history': [],
 'output': 'The area of a square with sides of 2 units is 4 square units.'}

Perfect, the model has responded without accessing the configured knowledge database.

Now I will try with a question that is also not related to health.

In [22]:
agent_executor.invoke({"input": "Do you know who is Clark Kent?"})



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: Medical KB
Action Input: Clark Kent I don't know.Do I need to use a tool? No
Final Answer: Clark Kent is the secret identity of Superman, a fictional superhero appearing in American comic books published by DC Comics. The character was created by writer Jerry Siegel and artist Joe Shuster, and first appeared in the comic book Action Comics #1 (cover-dated June 1938 and published April 18, 1938).

> Finished chain.


{'input': 'Do you know who is Clark Kent?',
 'chat_history': [HumanMessage(content='Give me the area of square of 2x2', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The area of a square with sides of 2 units is 4 square units.', additional_kwargs={}, response_metadata={})],
 'output': 'Clark Kent is the secret identity of Superman, a fictional superhero appearing in American comic books published by DC Comics. The character was created by writer Jerry Siegel and artist Joe Shuster, and first appeared in the comic book Action Comics #1 (cover-dated June 1938 and published April 18, 1938).'}

It has not accessed either, as the model has been able to identify that it is not a question related to the database that LangChain provides.

Now it's time to try with a question related to Medicine. Let's see if the model can understand that it should first look for information in the vector database at its disposal.

In [23]:
 agent_executor.memory.clear()

In [24]:
agent_executor.invoke({"input": """I have a patient that can have Botulism,
how can I confirm the diagnose?"""})



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: Medical KB
Action Input: Botulism Botulism is a rare but serious paralytic illness caused by a nerve toxin produced by certain bacteria. It can be contracted through contaminated food, wounds, or ingestion of bacterial spores. Symptoms include muscle paralysis, difficulty swallowing, and respiratory failure. Treatment includes antitoxin, supportive care, and removal of the source of the toxin.Do I need to use a tool? No
Final Answer: To confirm a diagnosis of botulism, a doctor may perform a physical exam, review symptoms and medical history, and order laboratory tests such as a stool or blood test. They may also perform a nerve conduction study or electromyography to assess muscle function. It is important to seek medical attention immediately if botulism is suspected.

> Finished chain.


{'input': 'I have a patient that can have Botulism,\nhow can I confirm the diagnose?',
 'chat_history': [],
 'output': 'To confirm a diagnosis of botulism, a doctor may perform a physical exam, review symptoms and medical history, and order laboratory tests such as a stool or blood test. They may also perform a nerve conduction study or electromyography to assess muscle function. It is important to seek medical attention immediately if botulism is suspected.'}

Perfect, the most important thing for us is that it has been able to identify that it should go to the medical database to search for information about the symptoms.

In [25]:
agent_executor.invoke({"input": "Is this an important illness?"})



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? No
Final Answer: Yes, botulism is a serious illness that requires immediate medical attention. It is caused by a toxin produced by the bacteria Clostridium botulinum and can lead to paralysis and even death if left untreated. It is important to seek medical attention immediately if botulism is suspected.

> Finished chain.


{'input': 'Is this an important illness?',
 'chat_history': [HumanMessage(content='I have a patient that can have Botulism,\nhow can I confirm the diagnose?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='To confirm a diagnosis of botulism, a doctor may perform a physical exam, review symptoms and medical history, and order laboratory tests such as a stool or blood test. They may also perform a nerve conduction study or electromyography to assess muscle function. It is important to seek medical attention immediately if botulism is suspected.', additional_kwargs={}, response_metadata={})],
 'output': 'Yes, botulism is a serious illness that requires immediate medical attention. It is caused by a toxin produced by the bacteria Clostridium botulinum and can lead to paralysis and even death if left untreated. It is important to seek medical attention immediately if botulism is suspected.'}

And the memory works perfectly. We can maintain a conversation, taking into account that the model knows the previous questions and answers.

## Evaluating the solution with Giskard

Install and Load the libraries.

In [26]:
!pip install -q giskard[llm]==2.15.1
from giskard.rag import KnowledgeBase, generate_testset, evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 17.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 544.9/544.9 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 88.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.8/140.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 113.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.3/143.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

Is necesary to create a Dataframe with just the column containing the information used to create the RAG system.

In [27]:
import pandas as pd
df_giskard = pd.DataFrame([d.page_content for d in df_document], columns=["text"])
df_giskard.head()

,text
0,LCMV infections can occur after exposure to fr...
1,LCMV is most commonly recognized as causing ne...
2,Individuals of all ages who come into contact ...
3,"During the first phase of the disease, the mos..."
4,"Aseptic meningitis, encephalitis, or meningoen..."


Using the information from the dataset, we ask Giskard to create a Knowledge Base, which is nothing more than a set of questions along with their respective answers. Both the questions and answers are generated by OpenAI's most advanced model, which is why it requires our OpenAI key to be provided.

In [28]:
kb_giskard = KnowledgeBase(df_giskard)

In [29]:
kb_giskard

In [30]:
# The more questions you generate, the more you will be charged.
test_questions = generate_testset(
    kb_giskard,
    num_questions=10,
    agent_description="Medical assistant for diagnosis and treatment support.",
)

INFO:giskard.rag:Finding topics in the knowledge base.
INFO:giskard.rag:Found 5 topics in the knowledge base.


Generating questions:   0%|          | 0/10 [00:00<?, ?it/s]

In [31]:
df_test_questions = test_questions.to_pandas()

In [32]:
df_test_questions.head()

,question,reference_answer,reference_context,conversation_history,metadata
id,,,,,
04ef0415-5c30-4265-941f-9d65790e9f34,What are some of the ways to prevent whipworm ...,"To prevent whipworm infection, avoid ingesting...",Document 15: The best way to prevent whipworm ...,[],"{'question_type': 'simple', 'seed_document_id'..."
4b435d83-6fa1-4ff9-bca6-9414971db308,What are the possible treatments for cysticerc...,There are medications available to treat cysti...,Document 9: Some people with cysticercosis do ...,[],"{'question_type': 'simple', 'seed_document_id'..."
83b3c29a-796d-4440-b38b-02a40662f506,In order to rule out other conditions similar ...,"Tests for botulism may include a brain scan, s...",Document 82: Physicians may consider the diagn...,[],"{'question_type': 'complex', 'seed_document_id..."
65c3ad44-0282-4212-b7a6-1d09becfec50,What are some specific actions that can be und...,Preventative measures include avoiding contact...,"Document 67: Raccoons are the primary, or defi...",[],"{'question_type': 'complex', 'seed_document_id..."
3ba564fe-936d-4836-bdfb-dbe0744bd2eb,Considering the context of body lice infestati...,The five main kinds of botulism are foodborne ...,"Document 80: In the United States, an average ...",[],"{'question_type': 'distracting element', 'seed..."


In [33]:
df_test_questions['question'].iloc[1]


'What are the possible treatments for cysticercosis?'

In [34]:
df_test_questions['reference_answer'].iloc[1]


'There are medications available to treat cysticercosis, and sometimes surgery may be needed.'

In [35]:
df_test_questions['reference_context'].iloc[1]

'Document 9: Some people with cysticercosis do not need to be treated. There are medications available to treat cysticercosis for those who do need treatment. Sometimes surgery may be needed. Your doctor will advise you on which treatment is best for you.\n    \nMore on: Resources for Health Professionals: Treatment\n    \nMore on: Taeniasis'

A function is created that will be called from Giskard's evaluate function. This function receives the question and returns the agent's response.

In [36]:
def use_agent(question, history=None):
  answer=agent_executor.invoke({"input": question})
  return answer['output']

In [37]:
agent_executor.memory.clear()

In [38]:
report = evaluate(use_agent, testset=test_questions, knowledge_base=kb_giskard)

Asking questions to the agent:   0%|          | 0/10 [00:00<?, ?it/s]



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: Medical KB
Action Input: Whipworm infection The best way to prevent whipworm infection is to avoid ingesting soil contaminated with human feces, wash hands before handling food, teach children about handwashing, and wash, peel, or cook all raw vegetables and fruits. Additionally, transmission of infection to others can be prevented by not defecating outdoors and having effective sewage disposal systems. Do I need to use a tool? No
Final Answer: Some ways to prevent whipworm infection include avoiding ingesting contaminated soil, washing hands before handling food, and properly washing, peeling, or cooking raw fruits and vegetables. It is also important to not defecate outdoors and have proper sewage disposal systems.

> Finished chain.


> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: Medical KB
Action Input: cysticercosis Cysticercosis is an infection caused by t

CorrectnessMetric evaluation:   0%|          | 0/10 [00:00<?, ?it/s]

In [39]:
# Summary with the results.
report.correctness_by_question_type()

,correctness
question_type,
complex,0.0
conversational,0.0
distracting element,0.5
double,0.0
simple,1.0
situational,0.5


In [40]:
report.correctness_by_topic()

,correctness
topic,
Infectious Diseases and Prevention,0.00
Others,0.20
Parasitic Diseases Treatment,0.75


In [41]:
report.component_scores()

,score
RAG Components,
GENERATOR,0.400000
RETRIEVER,0.750000
REWRITER,0.166667
ROUTING,1.000000
KNOWLEDGE_BASE,0.250000


In [42]:
# Obtaining the incorrect answers
failures = report.get_failures()[:2]
failures

,question,reference_answer,reference_context,conversation_history,metadata,agent_answer,correctness,correctness_reason
id,,,,,,,,
83b3c29a-796d-4440-b38b-02a40662f506,In order to rule out other conditions similar ...,"Tests for botulism may include a brain scan, s...",Document 82: Physicians may consider the diagn...,[],"{'question_type': 'complex', 'seed_document_id...",It is important for a physician to conduct the...,False,The agent did not provide the specific diagnos...
65c3ad44-0282-4212-b7a6-1d09becfec50,What are some specific actions that can be und...,Preventative measures include avoiding contact...,"Document 67: Raccoons are the primary, or defi...",[],"{'question_type': 'complex', 'seed_document_id...",Some specific actions that can be undertaken t...,False,The agent's answer is partially correct but it...


In [43]:
# Giskard explains the reasons why it considers the answers to be incorrect.
failures['correctness_reason'].iloc[1]

"The agent's answer is partially correct but it misses out on several important preventative measures such as washing hands after working or playing outdoors, not keeping or feeding wild animals as pets, discouraging raccoons from living in and around homes or parks, and the specific precautions to take when cleaning a latrine."


# Conclusions.
The experiment has been a small success. The Vectorial database has been configured and filled with information from the dataset. A LangChain agent has been created, and it has been able to retrieve information from the database only when necessary. Don't forget that our ChatBot has memory.

All of this in just a few lines of code!


---